In [1]:
# If you are using different method to initialize spark session, this block might be different.
import findspark
findspark.init()

import re
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

sc = pyspark.SparkContext(appName="hw5")

In [2]:
# Read data
data = sc.textFile('web-Google.txt').filter(lambda x: x[0]!='#').map(lambda x: re.split(r'\t+', x))

In [3]:
# Verify the data
data.take(3)

[['0', '11342'], ['0', '824020'], ['0', '867923']]

In [4]:
# define the class for hw5
class QueryRDD():
    def __init__(self, rdd):
        self.rdd = rdd
    def list_in_degree(self):
        return self.rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1], False)
    def list_out_degree(self):
        return self.rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1], False)
    def show_out_node(self, node):
        out_node = self.rdd.filter(lambda x: x[0] == node).map(lambda x: x[1])
        return out_node
    def show_in_node(self, node):
        in_node = self.rdd.filter(lambda x: x[1] == node).map(lambda x: x[0])
        return in_node

In [5]:
rdd = QueryRDD(data)

In [6]:
# Save the result of (1) and show some data
rdd.list_out_degree().coalesce(1).saveAsTextFile('out_degree')
rdd.list_out_degree().take(5)

[('506742', 456),
 ('203748', 372),
 ('305229', 372),
 ('768091', 330),
 ('808643', 277)]

In [7]:
# Save the result of (2) and show some data
rdd.list_in_degree().coalesce(1).saveAsTextFile('in_degree')
rdd.list_in_degree().take(5)

[('537039', 6326),
 ('597621', 5354),
 ('504140', 5271),
 ('751384', 5182),
 ('32163', 5097)]

In [8]:
#Sample result: list of nodes that node 506742 point to and show partial result
rdd.show_out_node('506742').coalesce(1).saveAsTextFile('506742_to')
rdd.show_out_node('506742').take(5)

['4999', '7248', '7544', '9082', '10298']

In [9]:
# Sample result: list of nodes that point to node 506742 and show partial result
rdd.show_in_node('506742').coalesce(1).saveAsTextFile('to_506742')
rdd.show_in_node('506742').take(5)

['163835', '630782', '402037', '745138', '810453']